In [1]:
# -*- coding: utf-8 -*-
'''
TopQuant-简称TQ极宽智能量化回溯分析系统，培训课件-配套教学python课件程序

Top极宽量化(原zw量化)，Python量化第一品牌 
by Top极宽·量化开源团队 2017.10.1 首发

网站： www.TopQuant.vip      www.ziwang.com
QQ群: Top极宽量化1群，124134140
      Top极宽量化2群，650924099
      Top极宽量化3群，450853713
  
'''
#1
import os,arrow
import pandas as pd
import numpy as np
#2
import keras
from keras import initializers,models,layers
from keras.models import Sequential
from keras.layers import Flatten,Dense, Input, Dropout, Embedding,SimpleRNN,Bidirectional,LSTM,Conv1D, GlobalMaxPooling1D,Activation,MaxPooling1D,GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras.utils import plot_model

#3
import tensorlayer as tl
import tensorflow as tf

#4
import zsys
import ztools as zt
import ztools_str as zstr
import ztools_data as zdat
import ztools_draw as zdr
import ztools_tq as ztq
import zpd_talib as zta
#
import zai_keras as zks

#
#------------------------------------

#1
print('\n#1,set.sys')
pd.set_option('display.width', 450)    
pd.set_option('display.float_format', zt.xfloat3)    
rlog='ailib/log_tmp'
if os.path.exists(rlog):tf.gfile.DeleteRecursively(rlog)

#2.1
print('\n#2.1,读取数据')
rss,fsgn,ksgn='ailib/TDS/','TDS2_sz50','avg'
xlst=zsys.TDS_xlst9
zt.prx('xlst',xlst)
#
df_train,df_test,x_train,y_train,x_test, y_test=zdat.frd_TDS(rss,fsgn,ksgn,xlst)
#
df_train,df_test,y_train,y_test=zdat.df_xed_xtyp2x(df_train,df_test,'3',k0=99.5,k9=100.5)
y_train,y_test=pd.get_dummies(df_train['y']).values,pd.get_dummies(df_test['y']).values
typ_lst=y_train[0]
num_in,num_out=len(xlst),len(typ_lst)
#
print('\ndf_test.tail()')
print(df_test.tail())
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))
#

#
#2.2
print('\n#2.2,转换数据格式shape')
rxn,txn=x_train.shape[0],x_test.shape[0]
x_train,x_test = x_train.reshape(rxn,num_in,-1),x_test.reshape(txn,num_in,-1)
print('\nx_train.shape,',x_train.shape)
print('\ntype(x_train),',type(x_train))


#
#3
print('\n#3,model建立神经网络模型')

print('\nnum_in,num_out:',num_in,num_out)
mx=zks.rnn010(num_in,num_out)
#
mx.summary()
plot_model(mx, to_file='tmp/rnn010.png')


#4 模型训练
print('\n#4 模型训练 fit')
tbCallBack = keras.callbacks.TensorBoard(log_dir=rlog,write_graph=True, write_images=True)
tn0=arrow.now()
mx.fit(x_train, y_train, epochs=500, batch_size=512,callbacks=[tbCallBack])
tn=zt.timNSec('',tn0,True)
mx.save('tmp/rnn010.dat')

#5 利用模型进行预测 predict
print('\n#5 模型预测 predict')
tn0=arrow.now()
y_pred0 = mx.predict(x_test)
tn=zt.timNSec('',tn0,True)
#
y_pred=np.argmax(y_pred0, axis=1)+1
df_test['y_pred']=zdat.ds4x(y_pred,df_test.index,True)
df_test.to_csv('tmp/df_rnn010.csv',index=False)

#============


Using TensorFlow backend.



#1,set.sys

#2.1,读取数据

 xlst
['open', 'high', 'low', 'close', 'volume', 'avg', 'ma_2', 'ma_3', 'ma_5', 'ma_10', 'ma_15', 'ma_20', 'ma_25', 'ma_30', 'ma_50', 'ma_100', 'xyear', 'xmonth', 'xday', 'xday_week', 'xday_year', 'xweek_year']

df_test.tail()
      open  high   low  close      volume   avg  ma_2  ma_3  ma_5  ma_10  ...    xmonth  xday  xday_week  xday_year  xweek_year     y  price  price_next  price_change  ktype
3065 3.280 3.290 3.270  3.290  858158.000 3.280 3.280 3.280 3.280  3.310  ...         1     1          3          1           0 3.000  3.280       3.310       100.915      3
3066 3.310 3.310 3.270  3.290 1654384.000 3.290 3.290 3.280 3.280  3.310  ...         1     1          3          1           0 3.000  3.290       3.310       100.608      3
3067 3.320 3.320 3.300  3.310 1210668.000 3.310 3.300 3.290 3.290  3.300  ...         1     1          3          1           0 2.000  3.310       3.310       100.000      2
3068 3.310 3.320 3.300  3.320  940023.000 3.310 3.310

120950/120950 [==============================] - 12s - loss: 1.7591 - acc: 0.6677    
Epoch 62/500
120950/120950 [==============================] - 12s - loss: 1.7621 - acc: 0.6626    
Epoch 63/500
120950/120950 [==============================] - 12s - loss: 1.7570 - acc: 0.6711    
Epoch 64/500
120950/120950 [==============================] - 11s - loss: 1.7425 - acc: 0.6700    
Epoch 65/500
120950/120950 [==============================] - 11s - loss: 1.7782 - acc: 0.6591    
Epoch 66/500
120950/120950 [==============================] - 13s - loss: 1.7435 - acc: 0.6683    
Epoch 67/500
120950/120950 [==============================] - 12s - loss: 1.7552 - acc: 0.6643    
Epoch 68/500
120950/120950 [==============================] - 12s - loss: 1.7517 - acc: 0.6681    
Epoch 69/500
120950/120950 [==============================] - 14s - loss: 1.7458 - acc: 0.6662    
Epoch 70/500
120950/120950 [==============================] - 13s - loss: 1.7424 - acc: 0.6674    
Epoch 71/500
120950/120

120950/120950 [==============================] - 13s - loss: 1.6928 - acc: 0.6657    
Epoch 144/500
120950/120950 [==============================] - 12s - loss: 1.6829 - acc: 0.6802    
Epoch 145/500
120950/120950 [==============================] - 12s - loss: 1.7171 - acc: 0.6707    
Epoch 146/500
120950/120950 [==============================] - 12s - loss: 1.6932 - acc: 0.6695    
Epoch 147/500
120950/120950 [==============================] - 12s - loss: 1.6881 - acc: 0.6652    
Epoch 148/500
120950/120950 [==============================] - 14s - loss: 1.6966 - acc: 0.6710    
Epoch 149/500
120950/120950 [==============================] - 12s - loss: 1.6858 - acc: 0.6720    
Epoch 150/500
120950/120950 [==============================] - 12s - loss: 1.6970 - acc: 0.6746    
Epoch 151/500
120950/120950 [==============================] - 13s - loss: 1.7118 - acc: 0.6743    
Epoch 152/500
120950/120950 [==============================] - 12s - loss: 1.7104 - acc: 0.6744    
Epoch 153/500


120950/120950 [==============================] - 11s - loss: 1.6378 - acc: 0.6752    
Epoch 225/500
120950/120950 [==============================] - 11s - loss: 1.6557 - acc: 0.6745    
Epoch 226/500
120950/120950 [==============================] - 11s - loss: 1.6514 - acc: 0.6778    
Epoch 227/500
120950/120950 [==============================] - 11s - loss: 1.6627 - acc: 0.6738    
Epoch 228/500
120950/120950 [==============================] - 11s - loss: 1.6389 - acc: 0.6721    
Epoch 229/500
120950/120950 [==============================] - 11s - loss: 1.6806 - acc: 0.6729    
Epoch 230/500
120950/120950 [==============================] - 12s - loss: 1.6452 - acc: 0.6749    
Epoch 231/500
120950/120950 [==============================] - 14s - loss: 1.6680 - acc: 0.6704    
Epoch 232/500
120950/120950 [==============================] - 12s - loss: 1.6482 - acc: 0.6750    
Epoch 233/500
120950/120950 [==============================] - 11s - loss: 1.6533 - acc: 0.6717    
Epoch 234/500


120950/120950 [==============================] - 12s - loss: 1.6372 - acc: 0.6724    
Epoch 305/500
120950/120950 [==============================] - 13s - loss: 1.6239 - acc: 0.6730    
Epoch 306/500
120950/120950 [==============================] - 13s - loss: 1.6100 - acc: 0.6724    
Epoch 307/500
120950/120950 [==============================] - 13s - loss: 1.6101 - acc: 0.6751    
Epoch 308/500
120950/120950 [==============================] - 13s - loss: 1.6234 - acc: 0.6731    
Epoch 309/500
120950/120950 [==============================] - 12s - loss: 1.6194 - acc: 0.6712    
Epoch 310/500
120950/120950 [==============================] - 13s - loss: 1.6159 - acc: 0.6769    
Epoch 311/500
120950/120950 [==============================] - 12s - loss: 1.6284 - acc: 0.6798    
Epoch 312/500
120950/120950 [==============================] - 12s - loss: 1.6214 - acc: 0.6736    
Epoch 313/500
120950/120950 [==============================] - 13s - loss: 1.6145 - acc: 0.6737    
Epoch 314/500


120950/120950 [==============================] - 11s - loss: 1.6826 - acc: 0.6760    
Epoch 386/500
120950/120950 [==============================] - 11s - loss: 1.6723 - acc: 0.6745    
Epoch 387/500
120950/120950 [==============================] - 11s - loss: 1.6841 - acc: 0.6762    
Epoch 388/500
120950/120950 [==============================] - 11s - loss: 1.6832 - acc: 0.6756    
Epoch 389/500
120950/120950 [==============================] - 12s - loss: 1.6736 - acc: 0.6700    
Epoch 390/500
120950/120950 [==============================] - 11s - loss: 1.6959 - acc: 0.6719    - ETA: 0s - loss: 1.6987 -
Epoch 391/500
120950/120950 [==============================] - 11s - loss: 1.6949 - acc: 0.6664    
Epoch 392/500
120950/120950 [==============================] - 12s - loss: 1.6850 - acc: 0.6722    
Epoch 393/500
120950/120950 [==============================] - 12s - loss: 1.7070 - acc: 0.6728    
Epoch 394/500
120950/120950 [==============================] - 12s - loss: 1.6877 - acc:

120950/120950 [==============================] - 14s - loss: 1.7543 - acc: 0.6626    
Epoch 467/500
120950/120950 [==============================] - 15s - loss: 1.7455 - acc: 0.6702    
Epoch 468/500
120950/120950 [==============================] - 16s - loss: 1.7212 - acc: 0.6670    
Epoch 469/500
120950/120950 [==============================] - 15s - loss: 1.7342 - acc: 0.6737    
Epoch 470/500
120950/120950 [==============================] - 12s - loss: 1.7546 - acc: 0.6763    
Epoch 471/500
120950/120950 [==============================] - 14s - loss: 1.7466 - acc: 0.6688    
Epoch 472/500
120950/120950 [==============================] - 15s - loss: 1.7451 - acc: 0.6721    
Epoch 473/500
120950/120950 [==============================] - 15s - loss: 1.7472 - acc: 0.6680    
Epoch 474/500
120950/120950 [==============================] - 12s - loss: 1.7566 - acc: 0.6706    
Epoch 475/500
120950/120950 [==============================] - 13s - loss: 1.7576 - acc: 0.6638    
Epoch 476/500
